In [ ]:
from ophyd import EpicsMotor, sim, Device
from ophyd import Component as Cpt, FormattedComponent as FCpt
from ophyd.sim import make_fake_device, SynAxis, FakeEpicsSignal
from ophyd.signal import EpicsSignal
from ophyd.status import MoveStatus
from ophyd.device import DynamicDeviceComponent
from bluesky.plans import count, scan
from bluesky.plan_stubs import mv
from bluesky.preprocessors import SupplementalData
from collections import OrderedDict

In [ ]:
from bluesky import RunEngine
RE = RunEngine({})

RE

In [ ]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

RE.subscribe(bec)

0

In [ ]:
from databroker import Broker
db = Broker.named('temp')

RE.subscribe(db.insert)

Tiled version 0.1.0b29


1

In [ ]:
sd = SupplementalData()
RE.preprocessors.append(sd)
sd.baseline

[]

In [ ]:
class LookupPair(Device):
    pair_name = FCpt(EpicsSignal, "{self.prefix}{self.name_postfix}", kind="omitted")
    pair_val = FCpt(EpicsSignal, "{self.prefix}{self.val_postfix}", kind="omitted")

    
    
    def __init__(self, *args, name_postfix : str, val_postfix : str, **kwargs):
        self.name_postfix = name_postfix
        self.val_postfix = val_postfix
        super().__init__(*args, **kwargs)

FakeLookupPair = make_fake_device(LookupPair)

In [ ]:
class DynamicLookup(Device):
    defn = OrderedDict({
        "pair1": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.ONST", "val_postfix" : "Val:1-SP"}),
        "pair2": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.TWST", "val_postfix" : "Val:2-SP"}),
        "pair3": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.THST", "val_postfix" : "Val:3-SP"}),
        "pair4": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.FRST", "val_postfix" : "Val:4-SP"}),
        "pair5": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.FVST", "val_postfix" : "Val:5-SP"}),
        "pair6": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.SXST", "val_postfix" : "Val:6-SP"}),
        "pair7": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.SVST", "val_postfix" : "Val:7-SP"}),
        "pair8": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.EIST", "val_postfix" : "Val:8-SP"}),
        "pair9": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.NIST", "val_postfix" : "Val:9-SP"}),
        "pair10": (FakeLookupPair, "", {"name_postfix" : "Pos-Sel.TEST", "val_postfix" : "Val:10-SP"})
    })

    lookupPairs = DynamicDeviceComponent(defn)


FakeDynamicLookup = make_fake_device(DynamicLookup)

In [ ]:
class MotorWithLookup(Device):
    
    pos_lookup = Cpt(FakeDynamicLookup, "")
    motor = FCpt(SynAxis, name = "motor", prefix = "{prefix}Mtr")
    pos = FCpt(FakeEpicsSignal, "", kind="hinted")

    def __init__(self, prefix : str, name = ""):
        super().__init__(prefix, name=name)

    def lookup(self, name: str) -> float:
        pair_lst = list(self.pos_lookup.lookupPairs.get())
        for pair in pair_lst:
            if pair.pair_name == name:
                return pair.pair_val       
        raise ValueError (f"Could not find {name} in lookup table")
    
    
    def get_all_positions(self):
        pair_lst = list(self.pos_lookup.lookupPairs.get())
        length = len(pair_lst)
        print(f"{length} possible positions:")
        print("----------------------------------")
        for pair in pair_lst:
            print(f'    {pair.pair_name:_<15} : {pair.pair_val}')

    
    def set_pos(self, pos: str | float):
        pair_lst = list(self.pos_lookup.lookupPairs.get())
        val = pos
        if isinstance(val, float):
            for pair in pair_lst:
                if pair.pair_val == val:
                    val = pair.pair_name
        return self.pos.set(str(val))


    def set(self, pos: str | float):
        self.set_pos(pos)
        if isinstance(pos, str):
            val = self.lookup_by_name(pos)
        else:
            val = pos
        return self.motor.set(val)

In [ ]:
my_motor = MotorWithLookup("my_motor", "motor:")

sd.baseline.append(my_motor)

my_motor.pos_lookup.lookupPairs.pair1.pair_name.put("out")
my_motor.pos_lookup.lookupPairs.pair1.pair_val.put(-55.0)
my_motor.pos_lookup.lookupPairs.pair2.pair_name.put("YAG")
my_motor.pos_lookup.lookupPairs.pair2.pair_val.put(-62.0)
my_motor.pos_lookup.lookupPairs.pair3.pair_name.put("Cu Block")
my_motor.pos_lookup.lookupPairs.pair3.pair_val.put(-74.0)
my_motor.pos_lookup.lookupPairs.pair4.pair_name.put("SrTiO3")
my_motor.pos_lookup.lookupPairs.pair4.pair_val.put(-76.0)
my_motor.pos_lookup.lookupPairs.pair5.pair_name.put("HEO-dark")
my_motor.pos_lookup.lookupPairs.pair5.pair_val.put(-79.0)
my_motor.pos_lookup.lookupPairs.pair6.pair_name.put("HEO-light")
my_motor.pos_lookup.lookupPairs.pair6.pair_val.put(-82.0)
my_motor.pos_lookup.lookupPairs.pair7.pair_name.put("MgAl2O4")
my_motor.pos_lookup.lookupPairs.pair7.pair_val.put(-86.0)
my_motor.pos_lookup.lookupPairs.pair8.pair_name.put("Si")
my_motor.pos_lookup.lookupPairs.pair8.pair_val.put(-89.0)
my_motor.pos_lookup.lookupPairs.pair9.pair_name.put("ZnSe")
my_motor.pos_lookup.lookupPairs.pair9.pair_val.put(-92.0)
my_motor.pos_lookup.lookupPairs.pair10.pair_name.put("Value 10")
my_motor.pos_lookup.lookupPairs.pair10.pair_val.put(0.0)

In [ ]:

RE(mv(my_motor, -50.0))

RE(scan([my_motor], my_motor, my_motor.lookup("out"), my_motor.lookup("Cu Block"), 5))



Transient Scan ID: 3     Time: 2025-06-25 13:50:16
Persistent Unique Scan ID: 'b4c03716-35ac-436c-9150-7a1844d0e5ee'
New stream: 'baseline'
Start-of-run baseline readings:
+--------------------------------+--------------------------------+
|                   motor:_motor | -50.0                          |
|                     motor:_pos | -50.0                          |
+--------------------------------+--------------------------------+
New stream: 'primary'
+-----------+------------+--------------+------------+
|   seq_num |       time | motor:_motor | motor:_pos |
+-----------+------------+--------------+------------+
|         1 | 13:50:16.9 |      -55.000 |        out |
|         2 | 13:50:16.9 |      -59.750 |     -59.75 |
|         3 | 13:50:17.0 |      -64.500 |      -64.5 |
|         4 | 13:50:17.0 |      -69.250 |     -69.25 |
|         5 | 13:50:17.1 |      -74.000 |   Cu Block |
+-----------+------------+--------------+------------+
generator scan ['b4c03716'] (scan num

('b4c03716-35ac-436c-9150-7a1844d0e5ee',)

In [ ]:
db[-1].table('baseline')

,motor:_motor,motor:_motor_setpoint,motor:_pos,time
seq_num,,,,
1,-50.0,-50.0,-50.0,2025-06-25 17:49:47.353407145
2,-74.0,-74.0,Cu Block,2025-06-25 17:49:47.656076431
